In [ ]:
#https://github.com/crux-toolkit/crux-toolkit/issues/278#issuecomment-292491088
# AUTHOR: William Stafford Noble
# CREATE DATE: 3 Mar 2017

import sys
def main():    
  global USAGE
  inputFile = "/home/animeshs/promec/Animesh/crux-3.1.Windows.AMD64/crux-output-new-bs3/search-for-xlinks.txt"

  # Parse the command line.
  inputFileName = inputFile

  # Grab and store the header.
  if (inputFileName == "-"):
    inputFile = sys.stdin
  else:
    inputFile = open(inputFileName, "r")
  header = inputFile.readline().rstrip()
  
  # Read the PSMs into memory.  In each case, the key is the entire
  # input line, and the value is a tuple of (score, decoyType), where
  # decoyType is "target", "target-decoy" or "decoy".
  (linears, intras, inters) = readPSMs(header, inputFile)
  
  # Estimate the FDRs separately.  After processing, target-decoy and
  # decoy-decoy PSMs are eliminated, and the (score, decoyType) tuples
  # are replaced by q-values.
  linears = computeQValues(linears, "linear")
  intras = computeQValues(intras, "intra-protein")
  inters = computeQValues(inters, "inter-protein")
  
  # Combine them all, then print.
  allPSMs = {}
  allPSMs.update(linears)
  allPSMs.update(intras)
  allPSMs.update(inters)
  printPSMs(header, allPSMs)
  
if __name__ == "__main__":
  main()
  

USAGE = """USAGE: compute-xlink-qvalue.py <PSMs>
  Compute q-values for cross-link peptide-spectrum matches, following
  the approach outlined in

     "False discovery rate estimation for crosslinked peptides
     identified by mass spectrometry" by Thomas Walzthoeni, Manfred
     Claassen, Alexander Leitner, Franz Herzog, Stefan Bohn, Friedrich
     Forster, Martin Beck & Ruedi Aebersold.  Nature
     Methods. 9(9):902-903, 2012.

  Input is a tab-delimited text file of PSMs, as produced by Crux
  search-for-xlinks with --concat=T and --top-match=1.  Use "-" to
  read from standard input.

  FDRs are estimated separately for three different types of PSMs:
  linears (including unlinked, mono-link, and loop-link peptides),
  intra-protein cross-links and inter-protein cross-links.  The latter
  category includes cross-links that occur both intra and inter
  (so-called "inter-intra" cross-links in Crux).

  For linear peptides, FDRs are estimated using standard target-decoy
  competition.  Note, however, that in this case and in the cross-link
  case, the estimated number of false positives is adjusted by adding
  1, so the FDR is (# decoys + 1) / (# targets).

  For the cross-linked peptides, the key idea is that every target is
  accompanied by three decoys: one in which both peptides are shuffled
  (decoy-decoy) and two in which a single peptide is shuffled
  (target-decoy and decoy-target).  In this setting, the expected
  number of false positives at a given score threshold is the total
  number of decoy PSMs above that threshold minus twice the number of
  decoy-decoy PSMs above that threshold.

  The program does the following steps:

   o Segregate the PSMs into three categories.

   o Estimate FDRs separately in each category.

   o Convert FDRs to q-values via minimization.

   o Merge all discoveries into a single list.

  Output is the same as the target input, with one additional column,
  "q-value."  The lines are sorted by increasing q-value.

"""   

######################################################################
# The protein ID column contains two protein IDs, separated by a
# semi-colon.  The names of the proteins begin with a reserved word if
# they are decoys.  This function returns "target", "target-decoy," or
# "decoy."
def getDecoyType(proteinID):

  decoyIndicator = "decoy_"

  ids = proteinID.split(";")
  # First, handle the linear case.
  if (len(ids) == 1):
    if (ids[0][:len(decoyIndicator)] == decoyIndicator):
      return("decoy")
    else:
      return("target")

  # Now handle the cross-linked case.
  else:
    (firstID, secondID) = ids
    if ((firstID[:len(decoyIndicator)] == decoyIndicator) and
        (secondID[:len(decoyIndicator)] == decoyIndicator)):
      return("decoy")
    elif ((firstID[:len(decoyIndicator)] == decoyIndicator) or
          (secondID[:len(decoyIndicator)] == decoyIndicator)):
      return("target-decoy")
    return("target")

######################################################################
def getColumnIndex(columnName, header):
  try:
    returnValue = header.split("\t").index(columnName)
  except:
    sys.stderr.write("Cannot find %s in header.\n%s\n"
                     % (columName, header))
    sys.exit(1)
  sys.stderr.write("Reading %s from column %d.\n"
                   % (columnName, returnValue + 1))

  return(returnValue)
  
######################################################################
# Read PSMs from an open file handle, segregating into three
# categories (linear, intra, inter).  PSMs are stored in dictionaries,
# where the key is the line from the file, and the value is a (score,
# decoy-type) tuple.
def readPSMs(header, inputFile):
  linears = {}
  intras = {}
  inters = {}

  # Figure out which column is which.
  scoreColIndex = getColumnIndex("xcorr score", header)
  typeColIndex = getColumnIndex("xlink type", header)
  proteinColIndex = getColumnIndex("protein id", header)

  # Read the file line by line.
  for line in inputFile:
    line = line.rstrip()
    words = line.split("\t")

    # Parse the line.
    score = float(words[scoreColIndex])
    psmType = words[typeColIndex]
    decoyType = getDecoyType(words[proteinColIndex])

    # Store the PSMs in the right place.
    if ((psmType == "linear") or (psmType == "dead-link")
        or (psmType == "self-loop")):
      linears[line] = (score, decoyType)
    elif (psmType == "xlink-intra"):
      intras[line] = (score, decoyType)
    elif ((psmType == "xlink-inter") or (psmType == "xlink-inter-intra")):
      inters[line] = (score, decoyType)
    else:
      sys.stderr.write("Unknown PSM type (%s).\n" % psmType)
      sys.exit(1)

  # Tell the user what's up.
  sys.stderr.write("Found %d linear, %d intra and %d inter cross-links.\n"
                   % (len(linears), len(intras), len(inters)))
  return(linears, intras, inters)

######################################################################
# Input is key=line, value=(score, type).
# Second indicates whether the peptides are linear.
# Output is key=line, value=q-value, but just including targets.
def computeQValues(psms, psmType):
  returnValue = {}

  if (len(psms) == 0):
    return(returnValue)

  # Tuple indices (for code readability)
  lineIndex = 0
  scoreIndex = 1
  isDecoyIndex = 2
  
  # Make a list of PSMs from the results.
  # List of lists: [line, score, decoyType]
  psmList = [] 
  for line in psms.keys():
    psmList.append([line, psms[line][0], psms[line][1]])

  # Sort the list by score.
  psmList.sort(key=lambda x: x[scoreIndex], reverse=True)
  sys.stderr.write("Computing q-values for %d %s PSMs.\n" % (len(psmList), psmType))

  # Compute FDRs.
  numTargets = 0
  numDecoys = 0
  numTargetDecoys = 0
  fdrs = []
  for index in xrange(0, len(psmList)):

    # Increment counters of targets and decoys.
    decoyType = psmList[index][isDecoyIndex]
    if (decoyType == "target"):
      numTargets += 1
    elif (decoyType == "decoy"):
      numDecoys += 1
    elif (decoyType == "target-decoy"):
      if (psmType == "linear"):
        sys.stderr.write("Invalid decoy type for linear peptide (%s).\n"
                         % decoyType)
      else:
        numTargetDecoys += 1

    # Estimate the number of false positives.
    if (psmType == "linear"):
      numFP = numDecoys
    else:
      numFP = max(0, numTargetDecoys - numDecoys)

    # Compute FDR, handling edge case.
    if (numTargets == 0):
      fdr = 1.0
    else: # Add one to avoid liberal bias.
      fdr = min(1.0, (numFP + 1.0) / numTargets)
    fdrs.append(fdr)
                   
  sys.stderr.write("  Found %d decoys" % numDecoys)
  if (psmType != "linear"):
    sys.stderr.write(", %d target-decoys," % numTargetDecoys)
  sys.stderr.write(" and %d targets.\n" % numTargets)

  # Handle ties. A block of ties should all be assigned the same FDR
  # as the last entry in the block.
  numTies = 0
  for index in xrange(len(psmList) - 2, 0, -1):
    if (psmList[index][scoreIndex] == psmList[index + 1][scoreIndex]):
      numTies += 1
      fdrs[index] = fdrs[index + 1]
  sys.stderr.write("  Found %d ties.\n" % numTies)
    
  # Compute q-values.
  minFDR = 1.0
  qValues = []
  for index in xrange(len(fdrs), 0, -1):
    if (fdrs[index - 1] < minFDR):
      minFDR = fdrs[index - 1]
    qValues = [minFDR] + qValues

  # Extract just the targets into a new list of PSMs.
  for index in xrange(0, len(fdrs)):
    if (psmList[index][isDecoyIndex] == "target"):
      returnValue[psmList[index][lineIndex]] = qValues[index]

  # Tell the user how many we accepted.
  for threshold in (0.01, 0.05, 0.1):
    numAccepted = 0
    for qValue in returnValue.values():
      if (qValue <= threshold):
        numAccepted += 1
    if (numAccepted > 1):
      sys.stderr.write("  Accepted %d PSMs at q<%g.\n"
                       % (numAccepted, threshold))

  return(returnValue)

######################################################################
# Input is key=line, value=(score, type).
# Second indicates whether the peptides are linear.
# Output is key=line, value=q-value, but just including targets.
def computeQValues2(psms, psmType):
  returnValue = {}

  if (len(psms) == 0):
    return(returnValue)

  # Tuple indices (for code readability)
  lineIndex = 0
  scoreIndex = 1
  isDecoyIndex = 2
  
  # Make a list of PSMs from the results.
  # List of lists: [line, score, decoyType]
  psmList = [] 
  for line in psms.keys():
    psmList.append([line, psms[line][0], psms[line][1]])

  # Sort the list by score.
  psmList.sort(key=lambda x: x[scoreIndex], reverse=True)
  sys.stderr.write("Computing q-values for %d %s PSMs.\n" % (len(psmList), psmType))

  # Compute FDRs, and put them in a new, target-only list.
  numTargets = 0
  numDecoys = 0
  numTargetDecoys = 0
  targetPSMList = [] # The new, target-only list. Entries are [line, qvalue].
  prevScore = 0.0
  numTies = 0
  for index in xrange(0, len(psmList)):

    # Increment counters of targets and decoys.
    decoyType = psmList[index][isDecoyIndex]
    if (decoyType == "target"):
      numTargets += 1
    elif (decoyType == "decoy"):
      numDecoys += 1
    elif (decoyType == "target-decoy"):
      if (psmType == "linear"):
        sys.stderr.write("Invalid decoy type for linear peptide (%s).\n"
                         % decoyType)
      else:
        numTargetDecoys += 1

    # Estimate the number of false positives.
    if (psmType == "linear"):
      numFP = numDecoys
    else:
      numFP = max(0, numTargetDecoys - numDecoys)

    # Compute FDR, handling edge case.
    if (numTargets == 0):
      fdr = 1.0
#      sys.stderr.write("%3d: FDR = 1. No targets.\n" % index)
    else: # Add one to avoid liberal bias.
      fdr = min(1.0, (numFP + 1.0) / numTargets)
#      sys.stderr.write("%3d: FDR = min(1, (%d + 1) / %d) = %g\n"
#                       % (index, numFP, numTargets, fdr))

    # Count ties.
    if (psmList[index][scoreIndex] == prevScore):
      numTies += 1
    prevScore = psmList[index][scoreIndex]

    # Only store target PSMs.
    if (decoyType == "target"):
      targetPSMList.append([psmList[index][lineIndex], fdr])

  # Report back to the user.
  sys.stderr.write("  Found %d decoys" % numDecoys)
  if (psmType != "linear"):
    sys.stderr.write(", %d target-decoys," % numTargetDecoys)
  sys.stderr.write(" and %d targets.\n" % numTargets)
  if (numTies > 0):
    sys.stderr.write("  Found %d ties.\n" % numTies)

  # Calculate q-values from FDRs.
  minFDR = 1.0
  for index in xrange(len(targetPSMList), 0, -1):
    (line, fdr) = targetPSMList[index - 1]
    if (fdr < minFDR):
      minFDR = fdr

    # Put it into the data structure.
    returnValue[line] = minFDR
    sys.stdout.write("2 Minimum FDR = %g\n" % minFDR)
  sys.stderr.write("  Minimum FDR = %g\n" % minFDR)

  # Tell the user how many we accepted.
  for threshold in (0.01, 0.05, 0.1):
    numAccepted = 0
    for qValue in returnValue.values():
      if (qValue <= threshold):
        numAccepted += 1
    if (numAccepted > 1):
      sys.stderr.write("  Accepted %d PSMs at q<%g.\n" % (numAccepted, threshold))

  return(returnValue)

######################################################################
# Input is a header string, and a dictionary of the form key = line,
# value = q-value.
def printPSMs(header, allPSMs):
  sys.stdout.write("%s\tq-value\n" % header)
  sortedTuples = sorted(allPSMs.items(), key=lambda x:x[1])
  sys.stderr.write("Printing %d PSMs.\n" % len(sortedTuples))
  for myTuple in sortedTuples:
    sys.stdout.write("%s\t%g\n" % myTuple)

######################################################################
# MAIN
######################################################################
